In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

(example-problem-02)=
## Example Problem 02

For this variation, we will use the same simple, perfect recycle scheme illustrated above for this problem, but we'll replace the PFR with a CSTR. 

```{figure} ../Images/CSTRRecycle.jpg
---
scale: 50%
name: CSTRRecycle
---
Separation with recycle for propane dehydrogenation (CSTR)
```

We will modify the problem in one additional way to allow a more straightforward solution: we will switch to a much simpler reaction--the irreversible conversion of $A$ into $B$:

$$A \longrightarrow B$$

This reaction occurs in the gas phase, it has a rate constant $k = 10 \ \textrm{s}^{-1}$ and we run it without a diluent, i.e., we feed Pure species $A$ into the process at a rate of $100 \ \mathrm{mol \ s^{-1}}$. The entire process is carried out at T = 573K and P = 1.0 atm. If the volume of the CSTR is $V = 400 \ \mathrm{L}$, and we recycle 50% of the unreacted species $A$, calculate the single pass and overall conversion of $A$ achieved in the process.

This problem is considerably more difficult because we cannot directly calculate the flowrates of $A$ coming out of the splitter, so we end up needing to solve a large system of algebraic equations simultaneously...

In [22]:
################################################################################
# Defining global constants for Example Problem 02                             #
################################################################################

FA0 = 100     #moles per second
k   = 10      #1/sec
T   = 573     #K
P   = 1.0     #atm
R   = 0.08206 #L*atm/mol/K
V   = 400     #L
α   = 0.5

################################################################################
# Define the system of algebraic equations that describes the Process          #
# It has the form f([FA1, FA2, FA3, FA4, FA5])                                 #
# It returns the LHS of the 5 algebraic equations                              # 
################################################################################

def P02(var):
    FA1, FA2, FA3, FA4, FA5 = var      
    Q1  = FA1*R*T/P #volumetric flowrate into CSTR
    Q2  = Q1        #A --> B so Q1 = Q2
    CA2 = FA2/Q2    #Concentration of A in CSTR
    r   = k*CA2;    #rate of reaction in CSTR
    RA2 = -r        #Production rate of A in CSTR
    
    LHS1 = FA0 + FA5 - FA1
    LHS2 = FA1 - FA2 + RA2*V
    LHS3 = FA2 - FA3
    LHS4 = FA3 - FA5 - FA4
    LHS5 = FA5 - α*FA3
    return [LHS1, LHS2, LHS3, LHS4, LHS5]

#################################################################################
# Provide initial guesses for FA1, FA2, FA3, FA4, FA5 when 0 = RHS              #
#################################################################################

var_guess = (150, 50, 50, 50, 50)

#################################################################################
# Solve the system of equations using opt.root()                                #
#################################################################################

ans2 = opt.root(P02, var_guess)

#################################################################################
# Extract answers for FA1, FA2, FA3, FA4, and FA5                               #
#################################################################################

FA1, FA2, FA3, FA4, FA5 = ans2.x

#################################################################################
# Calculate requested quantities for solution with α = 0.5 and VCSTR = 400L     #
#################################################################################

XSP    = (FA1 - FA2)/FA1    #single pass conversion A
XOV    = (FA0  - FA4)/FA0   #overall conversion of A
alpha  = FA5/FA3            #recycle ratio
print(f'XSP = {XSP:0.3f}, XOV = {XOV:0.3f}, and α = {α:0.2f}')

XSP = 0.368, XOV = 0.538, and α = 0.50


(example-problem-03)=
## Problem 03

Now let's make one small modification to [Example Problem 02](#example-problem-02): we will swap the CSTR out for a Plug Flow reactor. The remaining process and reactor specifications are the same. We are still considering the irrerversible conversion of $A$ into $B$:

$$A \longrightarrow B$$

The reaction has a rate constant $k = 10 \ \textrm{s}^{-1}$, and we run it without a diluent. We are feeding species A in at a rate of 100 moles per second. The entire process is carried out at T = 573K and P = 1.0 atm. If the volume of the PFR is 400L, and we recycle 50% of the unreacted species A, calculate the single pass and overall conversion of A achieved in this process.

```{figure} ../Images/PFRRecycle.jpg
---
scale: 50%
name: PFRRecycle
---
Separation with recycle for propane dehydrogenation (PFR)
```

```{note}
We take a very similar approach to the one we used in the [Solution to Example Problem 02](#solution-to-example-problem-02). As above, this will result in a system of equations comprising four material balances (one on each unit operation) and one specification about the recycle ratio $(\alpha = 0.5)$. We find an additional twist here in that one of our balances is a differential equation, so we can't avoid a bit of calculus in this solution.
```

In [23]:
################################################################################
# Define the system of algebraic equations that describes the Process          #
# It has the form f([FA1, FA2, FA3, FA4, FA5])                                 #
# It returns the LHS of the 5 algebraic equations                              # 
################################################################################

def P03(var):
    FA1, FA2, FA3, FA4, FA5 = var
    
    Q1   = FA1*R*T/P;     
    LHS1 = FA0 + FA5 - FA1
    LHS2 = FA2 - FA1*np.exp(-k/Q1*V)
    LHS3 = FA2 - FA3
    LHS4 = FA3 - FA5 - FA4
    LHS5 = FA5 - α*FA3
    return [LHS1, LHS2, LHS3, LHS4, LHS5]

#################################################################################
# Provide initial guesses for FA1, FA2, FA3, FA4, FA5 when 0 = RHS              #
#################################################################################

var_guess = (150, 50, 50, 50, 50)

#################################################################################
# Solve the system of equations using opt.root()                                #
#################################################################################

ans3 = opt.root(P03, var_guess)

#################################################################################
# Extract answers for FA1, FA2, FA3, FA4, and FA5                               #
#################################################################################

FA1, FA2, FA3, FA4, FA5 = ans3.x

#################################################################################
# Calculate requested quantities for solution with α = 0.5 and VPFR = 400L     #
#################################################################################

XSP    = (FA1 - FA2)/FA1    #single pass conversion A
XOV    = (FA0  - FA4)/FA0   #overall conversion of A
alpha  = FA5/FA3            #recycle ratio
print(f'XSP = {XSP:0.3f}, XOV = {XOV:0.3f}, and α = {α:0.2f}')

XSP = 0.463, XOV = 0.633, and α = 0.50


## Problem 04

For the above case in [Example Problem 03](#example-problem-03), things aren't so bad despite the differential equation that shows up when we initially write balances. We have first order kinetics and a constant volumetric flowrate; further, we know the size of the reactor!. This means that the PFR balance can be solved analytically, and we can replace the differential equation with an algebraic one and proceed as normal with an equation solver.

What happens if we have to deal with more difficult kinetics?  For this example, let's stick with the simple reaction:

$$A \longrightarrow B$$

But we'll say that it has a complex rate law:

$$r = \frac{kC_A}{1+KC_A}$$

Where $k = 10 \ \textrm{s}^{-1}$ as before and $K = 17.5 \ \textrm{L} \ \textrm{mol}^{-1}$. Otherwise, the problem is identical. We are feeding species $A$ in at a rate of $100 \ \mathrm{mol \ s^{-1}}$. The entire process is carried out at $T = 573K$ and $P = 1.0 \ \mathrm{atm}$. If the volume of the PFR is 400L, and we recycle 50% of the unreacted species $A$, calculate the single pass and overall conversion of $A$ achieved in this process.

We still need to solve a system of algebraic equations as above, but the catch now is that the differential equation that describes the PFR balance is more cumbersome to solve. I don't know an analytical solution offhand. We *could* develop one after separating variables. That said, in my opinion, it will be easier and cleaner to use numerical methods to integrate it. For this solution, we'll use `solve_ivp()` because it is relatively easy to implement a Runge-Kutta method here since we know the upper limit of integration (V = 400L). You could also do this with Gaussian Quadrature, but I prefer the `solve_ivp()` approach as it is easier to extend to more difficult systems that involve several coupled ODEs. This is an important technique to learn because you will not always be able to solve ODEs analytically.

In [24]:
################################################################################
# Defining new global constants for Example Problem 04                         #
################################################################################

K   = 17.5    #mol/L

In [25]:
################################################################################
# Define the system of algebraic equations that describes the Process          #
# It has the form f([FA1, FA2, FA3, FA4, FA5])                                 #
# It returns the LHS of the 5 algebraic equations                              # 
################################################################################

def P04(var):
    FA1, FA2, FA3, FA4, FA5 = var

    Q1  = FA1*R*T/P

    vspan = (0, V)
    var0  = FA1
    ans   = solve_ivp(odemodel, vspan, [var0], args = (Q1, )) #To determine the exit flowrate of the PFR, we need to solve an ODE...
    FPFR  = ans.y[0,-1]
      
    LHS1 = FA0 + FA5 - FA1
    LHS2 = FA2 - FPFR
    LHS3 = FA2 - FA3
    LHS4 = FA3 - FA5 - FA4
    LHS5 = FA5 - α*FA3
    return [LHS1, LHS2, LHS3, LHS4, LHS5]

################################################################################
# Define the ODE that describes the PFR                                        #
# It has the form f(V, [FA], Q1)                                               #
# It returns [dFAdV]                                                           # 
# Q1 is passed as a parameter because it depends on FA1, which is unknown      #
# The algebraic solver is varying FA1, so Q1 changes on each pass              #
################################################################################

def odemodel(vol, var, par):
    FA = var
    Q1 = par
    CA = FA/Q1
    D  = -k*CA/(1 + K*CA)
    return D

#################################################################################
# Provide initial guesses for FA1, FA2, FA3, FA4, FA5 when 0 = RHS              #
#################################################################################

var_guess = (150, 50, 50, 50, 50)

#################################################################################
# Solve the system of equations using opt.root()                                #
#################################################################################

ans4 = opt.root(P04, var_guess)

#################################################################################
# Extract answers for FA1, FA2, FA3, FA4, and FA5                               #
#################################################################################

FA1, FA2, FA3, FA4, FA5 = ans4.x

#################################################################################
# Calculate requested quantities for solution with α = 0.5 and VPFR = 400L     #
#################################################################################

XSP    = (FA1 - FA2)/FA1
XOV    = (FA0  - FA4)/FA0
alpha  = FA5/FA3
print(f'XSP = {XSP:0.3f}, XOV = {XOV:0.3f}, and α = {α:0.2f}')

XSP = 0.359, XOV = 0.528, and α = 0.50
